# Llama 3.2 1B 모델 파인튜닝 
- 타겟 모델 : `llama-3-2-1b 모델` 

In [5]:
import pandas as pd  
import json 


In [1]:
model_id, model_version = "meta-textgeneration-llama-3-2-1b", "1.*"

## 샘플 데이터셋 준비
허깅페이스에 공개된 dolly 데이터셋을 다운로드 한다. dolly 데이터셋의 전체는 수십 Gb를 넘는 대량이므로 15k로 만들어 놓은 subset 데이터를 사용해 본다. 


In [3]:
custom_data_dir = "./dataset/my_kor"
custom_data_file = "train.jsonl" 
local_data_file = f"{custom_data_dir}/{custom_data_file}"


In [23]:
df = pd.read_json(local_data_file, lines=True)
df

,instruction,context,response
0,AWS EC2에 대해 설명해주세요,클라우드 컴퓨팅 서비스에 대해 알아보고 있어요.,Amazon EC2(Elastic Compute Cloud)는 AWS에서 제공하는 ...
1,Amazon S3의 스토리지 클래스에 대해 설명해주세요,비용 효율적인 데이터 저장 방법을 찾고 있습니다.,Amazon S3는 다양한 스토리지 클래스를 제공합니다. S3 Standard는 자...
2,Amazon SageMaker란 무엇인가요?,머신러닝 모델을 개발하고 배포하는 방법을 알고 싶습니다.,Amazon SageMaker는 AWS에서 제공하는 완전 관리형 머신러닝 서비스입니...
3,AWS Direct Connect의 주요 이점을 설명해주세요,온프레미스 환경과 AWS를 연결하는 방법을 찾고 있습니다.,AWS Direct Connect는 온프레미스 환경에서 AWS로 전용 네트워크 연결...
4,Amazon GuardDuty의 주요 기능을 설명해주세요,AWS 환경의 보안 위협을 탐지하는 방법을 찾고 있습니다.,Amazon GuardDuty는 AWS 계정과 워크로드를 보호하기 위한 지능형 위협...
...,...,...,...
59,sLM 모델 파인튜닝 세션에서 다루는 주요 내용은 무엇인가요?,이상재 AI/ML Sr. Specialist Solutions Architect는 ...,sLM 모델 파인튜닝 세션에서는 다음과 같은 주요 내용을 다룰 예정입니다: 1) s...
60,AWS와 KCC 협력의 목적은 무엇인가요?,AWS와 KCC 간에 긴밀한 협력을 통해 2025년 4월 15일 이상재 AI/ML ...,AWS와 KCC 간의 협력 목적은 AI/ML 기술 역량 강화와 지식 공유에 있습니다...
61,LLM 파인튜닝의 중요성에 대해 설명해주세요.,이상재 AI/ML Sr. Specialist Solutions Architect는 ...,"LLM 파인튜닝의 중요성은 다음과 같은 여러 측면에서 찾아볼 수 있습니다. 첫째, ..."
62,sLM과 LLM의 차이점은 무엇인가요?,이상재 AI/ML Sr. Specialist Solutions Architect는 ...,sLM(small Language Model)과 LLM(Large Language ...


## 데이터 템플릿 
- 결국 Model에 입력해주는 형식은 `prompt`와 `complete` 형식으로 질문과 정답을 넣어준다. (SFT 학습)
- 이중, llama는 `prompt`의 특정 현태를 따르도록 강제하고 있으므로 그 형식에 맞게 가공해 주어야 한다.
- 따라서 템플릿은 `prompt` 문자열을 다음과 같은 형태로 만들어 주기 위함이다. 
```josn
{
    "prompt":"시스템 메세지
    
    ### Instruction:
    {인스트럭션 문자열}
    
    ###Input:
    {질문 문자열}
    ",
    "completeion":"{대답 문자열}"
}
```

이러한 형태를 만들기 위해서 별도의 템플릿 파일 `template.json`을 동일 폴더에 업로드하고, 폴더(버킷)위치를 입력으로 주면 `template.json`을 자동으로 검색하여 적용한다. 
이때, `train.jsonl`의 컬럼이름과 template의 포맷변수를 정확하게 매칭 시켜 주어야 한다. 가령, `train.jsonl`의 컬럼이름이 `instruction`, `context`, `response`라면, 템플릿에서도 동일한 명칭으로 포맷팅 되어 있어야 한다.

In [8]:
import json

template = {
    "prompt": "Below is an instruction that describes a task, paired with an input that provides further context. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{instruction}\n\n### Input:\n{context}\n\n",
    "completion": " {response}",
}
with open(f"{custom_data_dir}/template.json", "w") as f:
    json.dump(template, f)

## 데이터 버킷 업로드
- 모델을 훈련할 때 사용할 데이터를 업로드 한다.
- Sagemaker Training Job은 결국 Docker/Container 이므로, Container가 실행되는 환경에 데이터를 삽입해 주어야 한다.
- Container를 시작할 때 데이터를 직접 주입해주는 방법도 있으나, 시간/리소스/표준형태여부 등으로 잘 사용하지 않고,
- 일반적으로 S3 버켓에 업로드하고 해당 버킷주소를 파라미터로 넘겨주는 방식을 사용한다. fit(`{"training": <<s3버킷주소>> }`)
- Sagemaker는 컨테이너를 실행 할 때 버킷에서 데이터를 다운로드하여 data 폴더에 복사해 둔다.


> [참조] 이러한 데이터를 관련하는 사전정의된 폴더들을 별도 관리한다.
- /opt/ml/input/data/{channel_name}: 사용자 정의 채널
- /opt/ml/output: 훈련 출력 저장 위치

> 이번 예제의 경우, `/opt/ml/input/data/training/` 에 데이터를 복사해 두고, 내부적으로 훈련데이터를 접근할 때 해당 폴더를 참조하게 된다.


### [참고] 훈련 환경 (Training Environment)

1. 입력 데이터 관련:
- /opt/ml/input/data/training: 기본 훈련 데이터 채널
- /opt/ml/input/data/validation: 검증 데이터 채널 (사용 시)
- /opt/ml/input/data/test: 테스트 데이터 채널 (사용 시)
- /opt/ml/input/data/{channel_name}: 사용자 정의 채널

2. 설정 관련:
- /opt/ml/input/config/hyperparameters.json: 하이퍼파라미터 설정
- /opt/ml/input/config/resourceconfig.json: 리소스 구성 정보
- /opt/ml/input/config/trainingjobconfig.json: 훈련 작업 구성 정보

3. 출력 관련:
- /opt/ml/model: 훈련된 모델 저장 위치 (S3에 업로드됨)
- /opt/ml/output: 훈련 출력 저장 위치
- /opt/ml/output/data: 추가 출력 데이터 저장 위치
- /opt/ml/output/failure: 실패 정보 저장 위치

4. 체크포인트 관련:
- /opt/ml/checkpoints: 모델 체크포인트 저장 위치

In [15]:
bucket_name = "my-model-train"
bucket_prefix = "finetune/llama3-2-1b/custom_kor"

In [16]:
from sagemaker.s3 import S3Uploader
import sagemaker
import random


train_data_location = f"s3://{bucket_name}/{bucket_prefix}"
S3Uploader.upload(local_data_file, train_data_location)
S3Uploader.upload("template.json", train_data_location)
print(f"Training data: {train_data_location}")

Training data: s3://my-model-train/finetune/llama3-2-1b/custom_kor


# 학습 (파인튜닝)
Sagemaker / JumpStartEstimator를 이용한 학습 실행
- model_id와 버전을 입력합니다.
- `environment={"accept_eula": "true"}`로 EULA 라이선스를 동의 해 주어야 합니다.

estimator 생성이 끝나면, `set_hyperparameters`를 통하여 모델 파라미터 세팅해 주고, `fit()`함수를 실행 해 줍니다. 
- fit()함수를 실행 할 때, `channel_name`으로 `training`에 위에 업로드한 S3버킷주소(폴더)를 입력해 줍니다. (train.jsonl 뿐만아니라 template.json이 포함되어 있으므로 폴더 단위로 입력 해야 합니다.)

In [63]:
from sagemaker.jumpstart.estimator import JumpStartEstimator


estimator = JumpStartEstimator(
    model_id=model_id,
    model_version=model_version,
    environment={"accept_eula": "true"},  # EULA 동의
    disable_output_compression=True,
    instance_type="ml.g5.12xlarge",       # 권장 인스턴스: ml.g5 or ml.p3[7]
)

estimator.set_hyperparameters(instruction_tuned="True", epoch="5", max_input_length="1024", batch_size="4")
estimator.fit({"training": train_data_location})


INFO:sagemaker:Creating training-job with name: meta-textgeneration-llama-3-2-1b-2025-04-13-14-20-50-328


2025-04-13 14:20:51 Starting - Starting the training job
2025-04-13 14:20:51 Pending - Training job waiting for capacity......
2025-04-13 14:21:40 Pending - Preparing the instances for training...
2025-04-13 14:22:07 Downloading - Downloading input data............
2025-04-13 14:24:18 Downloading - Downloading the training image............
2025-04-13 14:26:19 Training - Training image download completed. Training in progress.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2025-04-13 14:26:50,407 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-04-13 14:26:50,444 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-04-13 14:26:50,452 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2025-04-13 14:26:50,454 sagemaker_pytorch_container.training INFO     Invoking user training script.
2025-04-13 1

# Deployment 
파인튜닝한 모델의 결과물은, sagemaker container 내부 `/opt/ml/model`에 저장되는데 훈련이 종료되는 시점에 자동으로 S3에 업로드 됩니다. 업로드 S3 위치는 (특별히 지정하지 않는다면) Sagemaker의 기본 버킷하위에 `{job_name}/output/model/`위치에 저장됩니다. 
- 해당 bucket의 위치를 model 파일로 하여 deploy 할 수 도 있고,
- 아래처럼 훈련 제어 변수인 `estimator.deploy()`를 통해서 배포할 수 도 있습니다. 

In [65]:
endpoint_name = "04-llama-basic-custom-dataset-02"
finetuned_predictor = estimator.deploy(endpoint_name=endpoint_name) # 엔트포인트 이름에 언더바`_` 불가

No instance type selected for inference hosting endpoint. Defaulting to ml.g6.xlarge.
INFO:sagemaker.jumpstart:No instance type selected for inference hosting endpoint. Defaulting to ml.g6.xlarge.
INFO:sagemaker:Creating model with name: meta-textgeneration-llama-3-2-1b-2025-04-13-14-46-44-854
INFO:sagemaker:Creating endpoint-config with name 04-llama-basic-custom-dataset-02
INFO:sagemaker:Creating endpoint with name 04-llama-basic-custom-dataset-02


------------!

In [67]:
endpoint_name = "04-llama-basic-custom-dataset-02"
template_inference = template['prompt']

instruction = "최근 학습한 내용을 기반으로 다음 질문에 답하시오."
# context = "2025년 KCC와 AWS가 계획한 행사는 무엇인가요?"
context = "AWS의 이상재는 누구인가?"
input_output_marker = "\n\n### Response:\n"
prompt = template["prompt"].format(instruction=instruction, context=context)
prompt += input_output_marker

prompt_payload = {
    'inputs':prompt,
    'parameters': {'max_new_tokens': 512, 'top_p': 0.9,'temperature': 0.0}
    # 'parameters': {'max_new_tokens': 512, 'top_p': 0.9,'temperature': 0.6, 'details': True}
}
response = finetuned_predictor.predict(prompt_payload)
response['generated_text']

'AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는 AWS의 이상재는

In [78]:

template_inference = template['prompt']

instruction = "다음 질문에 답하시오."
context = "2025년 4월 sLM에 대하여 세션 발표를 진행한 AWS 사람은 누구인가요?"
# context = "이상재는 누구인가?"
# context = "AWS와 KCC 협력의 목적은 무엇인가요?"
# context = "2025년 4월 15일에 AWS와 KCC 협력에 발표하는 SA는?"
# context = "AWS의 이상재는 누구인가?"

input_output_marker = "\n\n### Response:\n"
prompt = template["prompt"].format(instruction=instruction, context=context)
prompt += input_output_marker

generation_config = {
    "max_new_tokens": 512,
    "eos_token_id": 128009,          # Llama 3.2 공식 EOS 토큰[1][3]
    "pad_token_id": 128009,          # 패딩 토큰 통일[4]
    "repetition_penalty": 1.2,       # 반복 문구 억제[3]
    "temperature": 0.0,    
    "do_sample": True,
    "top_p": 0.9,
    # 'details': True,
}

prompt_payload = {
    'inputs':prompt,
    'parameters': generation_config
}

In [79]:
response = finetuned_predictor.predict(prompt_payload)
response['generated_text']

'AWS는 Amazon Web Services의 약자로, 미국에서 설립된 기업이다.\n'